In [1]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")


assert GEMINI_API_KEY, "GEMINI_API_KEY is missing. Check your .env file."
assert OPENAI_API_KEY, "OPENAI_API_KEY is missing. Check your .env file."
assert TAVILY_API_KEY, "TAVILY_API_KEY is missing. Check your .env file."

print("Key loaded:", GEMINI_API_KEY[:6] + "..." )
print("OpenAI Key loaded:", OPENAI_API_KEY[:6] + "...")
print("Tavily Key loaded:", TAVILY_API_KEY[:6] + "...")

# Initialize LLMs
gemini_llm = ChatGoogleGenerativeAI(
    temperature=0,
    model="gemini-2.5-flash",
    google_api_key=GEMINI_API_KEY
)
openai_llm = ChatOpenAI(
    temperature=0,
    model="gpt-3.5-turbo",
    openai_api_key=OPENAI_API_KEY
)

# Prompt template
prompt = PromptTemplate(
    input_variables=["product"],
    template="Give me a creative name for a company that makes {product}?",
)

# Example: Use Gemini
chain = prompt | gemini_llm
response = chain.invoke({"product": "smart home devices"})
print("Gemini response:", response)

# Example: Use OpenAI
chain = prompt | openai_llm
response = chain.invoke({"product": "smart home devices"})
print("OpenAI response:", response)
import langchain
print("LangChain version:", langchain.__version__)

Key loaded: AIzaSy...
OpenAI Key loaded: sk-pro...
Tavily Key loaded: tvly-d...
Gemini response: content='Here are some creative names for a smart home device company, categorized by their vibe:\n\n**Modern & Tech-Forward:**\n\n1.  **AuraSync:** (Suggests a harmonious, interconnected atmosphere)\n2.  **Verta Automation:** (From "vertex," implying peak control and innovation)\n3.  **HomeOS:** (Simple, bold, positions the company as the operating system for the home)\n4.  **Nexus Dwell:** (Nexus means a central link or connection; Dwell for living)\n5.  **CogniCasa:** (Cognitive + Casa (home in Spanish), implying intelligence within the home)\n6.  **LumenFlow:** (Lumen for light/brightness; Flow for seamless operation)\n7.  **Synapse Systems:** (Refers to brain connections, emphasizing intelligence and connectivity)\n8.  **Evolve Home:** (Suggests continuous improvement and adaptation)\n9.  **EchoSense:** (Plays on "echo" for responsiveness and "sense" for awareness)\n10. **Aether Living

In [6]:
from pydantic import BaseModel, Field
from langchain.agents import create_agent
from langchain.tools import tool

# ---- Args schemas ----

class SearchDatabaseArgs(BaseModel):
    query: str = Field(..., description="Query to search in the customer database")

class WeatherArgs(BaseModel):
    city: str = Field(..., description="City name to get weather for")

# ---- Tools with args_schema ----

@tool("search_database", args_schema=SearchDatabaseArgs)
def search_database(query: str) -> str:
    """Search the customer database for records."""
    return f"Found 3 results for '{query}'"

@tool("get_weather", args_schema=WeatherArgs)
def get_weather(city: str) -> str:
    """Get current weather for a city."""
    return f"Weather in {city}: Sunny, 72°F"

# ---- Agent (uses openai_llm from cell 1) ----

tools = [search_database, get_weather]

agent = create_agent(
    model=openai_llm,
    tools=tools,
    system_prompt="You are a helpful assistant with access to search and weather tools.",
)

# ---- Example call ----

result = agent.invoke({
    "messages": [
        {
            "role": "user",
            "content": "What's the weather in SF and any DB records for weather?",
        }
    ]
})
print(result["messages"][-1].content)

The weather in San Francisco is currently sunny with a temperature of 72°F. Additionally, there are 3 records found in the database related to weather.
